In [69]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
import torch
import torch.nn
import math

In [25]:
attri_data = pd.read_csv('../LiterallyWikidata/files_needed/train_attri_data_minmax.csv')
attri_data_valid = pd.read_csv('../LiterallyWikidata/files_needed/valid_attri_data_minmax.csv')
attri_data_test = pd.read_csv('../LiterallyWikidata/files_needed/test_attri_data_minmax.csv')

In [5]:
# with open('../LiterallyWikidata/files_needed/attribute.txt','r') as fr:
#     att2idx = {line.strip(): i for i, line in enumerate(fr)}
# with open('../LiterallyWikidata/files_needed/list_ent_ids.txt','r') as f:
#     ent2idx ={line.strip(): i for i, line in enumerate(f)}

In [34]:
## constraint needed:
pop_idx = att2idx['P1082']
gdp = att2idx['P4010']
nominal_gdp = att2idx['P2131']
nominal_gdp_per = att2idx['P2132']
gdp_per = att2idx['P2299']
date_of_birth = att2idx['P569']
date_of_death = att2idx['P570']
area = ['P2046']
net_profit = att2idx['P2295']
retirement_age = att2idx['P3001']
age_of_majority = att2idx['P2997']
work_start = att2idx['P2031']
work_end = att2idx['P2032']
gdp

KeyError: 'P2295'

In [26]:
# load pretrained e, make ebt2idx, map to ori data to get entid
emb_ent = torch.load('../LiterallyWikidata/files_needed/pretrained_kge/pretrained_complex_entemb.pt')
#print(emb_ent.shape)
ent2idx ={}
with open('../LiterallyWikidata/files_needed/list_ent_ids.txt','r') as fr:
    for i, word in enumerate(fr.readlines()):
        ent2idx[word.strip()] = i
embedding_e = torch.nn.Embedding.from_pretrained(emb_ent)
attri_data['e_idx']=attri_data['e'].map(ent2idx)
input_e = torch.LongTensor(attri_data['e_idx'].to_numpy())
emb_e =embedding_e(input_e)
# load emb a, make att2idx, map to ori data to get attid   
          
att2idx = {v:k for k,v in enumerate(attri_data['a'].unique())}
embedding_a = torch.nn.Embedding(len(attri_data['a'].unique()),128,padding_idx=0)
attri_data['a_idx']=attri_data['a'].map(att2idx)
input_a = torch.LongTensor(attri_data['a_idx'].to_numpy())
emb_a = embedding_a(input_a)
# 
# attri_data_valid['a_idx']=attri_data_valid['a'].map(att2idx)
# attri_data_valid['e_idx']=attri_data_valid['e'].map(ent2idx)     

In [20]:
list(attri_data['a'].unique()),list(attri_data_valid['a'].unique())

(['P625_Latitude',
  'P625_Longtiude',
  'P1332_Latitude',
  'P1082',
  'P2046',
  'P2067',
  'P2043',
  'P2031',
  'P576',
  'P569',
  'P1332_Longtiude',
  'P1538',
  'P571',
  'P1333_Latitude',
  'P2044',
  'P6543',
  'P6545',
  'P2927',
  'P1334_Latitude',
  'P6509',
  'P570',
  'P1350',
  'P1333_Longtiude',
  'P582',
  'P1334_Longtiude',
  'P6544',
  'P1335_Longtiude',
  'P2048',
  'P6547',
  'P1335_Latitude',
  'P2032',
  'P2243',
  'P3087',
  'P2052',
  'P577',
  'P2225',
  'P1619',
  'P1373',
  'P6546',
  'P2045',
  'P580',
  'P2437',
  'P1113',
  'P2047',
  'P1128',
  'P1132',
  'P2299',
  'P2244',
  'P2121',
  'P4511',
  'P2142',
  'P5982',
  'P4841',
  'P3157',
  'P5595',
  'P1103',
  'P2132',
  'P2219',
  'P1351',
  'P1249',
  'P1418',
  'P585',
  'P3086',
  'P1971',
  'P1540',
  'P2053',
  'P1352',
  'P1831',
  'P3872',
  'P2884',
  'P4010',
  'P1110',
  'P2131',
  'P2873',
  'P620',
  'P4080',
  'P619',
  'P2049',
  'P2250',
  'P2130',
  'P2234',
  'P2146',
  'P1083',
  'P

In [73]:
# attri_data_valid 做testdata
attri_data_valid['e_idx']=attri_data_valid['e'].map(ent2idx)
input_e_val = torch.LongTensor(attri_data_valid['e_idx'].to_numpy())
emb_e_val =embedding_e(input_e_val)*math.sqrt(2./128)
attri_data_valid['a_idx']=attri_data_valid['a'].map(att2idx)
input_a_val = torch.LongTensor(attri_data_valid['a_idx'].to_numpy())
emb_a_val = embedding_a(input_a_val)*math.sqrt(2./128)
x_val = torch.cat([emb_e_val,emb_a_val],dim=1).detach().numpy()
y_val= attri_data_valid.loc[:,'v'].values
X_test,y_test=x_val,y_val

In [6]:
attri_data[:5]

,e,a,v,name_e,name_a,ent_type,minmax
0,Q593336,P625_Latitude,4.315194e+01,Poumarous,coordinate location(latitude),Q484170,7.397330e-01
1,Q511127,P625_Longtiude,3.301111e+00,Nant,coordinate location(logtitude),Q484170,5.068894e-01
2,Q59293,P1332_Latitude,4.329426e+01,Porto Sant'Elpidio,coordinates of northernmost point,Q747074,6.482318e-01
3,Q100092,P1082,6.460000e+03,San Cesario sul Panaro,population,Q747074,1.551299e-06
4,Q111736,P2046,1.768000e+07,Sarezzo,area,Q747074,2.903120e-12


In [11]:
# ent embedding use not pretrain
# ent2idx = {e:i for i,e in enumerate(attri_data['e'].unique())}
# attri_data['e_idx']=attri_data['e'].map(ent2idx)
# embedding_e = torch.nn.Embedding(len(attri_data['e'].unique()),128,padding_idx=0)
# input_e = torch.LongTensor(attri_data['e_idx'].to_numpy())
# emb_e = embedding_e(input_e)

In [74]:
x = torch.cat([emb_e,emb_a],dim=1).detach().numpy()

In [75]:
x

array([[-3.1758689e-03, -3.0028114e-02,  1.9056829e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-8.0347735e-01, -1.6112080e-01,  1.5777169e+00, ...,
         1.4985690e-01,  1.9457540e-01,  7.9213321e-01],
       [-1.1778606e+00,  2.5393713e-02, -1.3152606e+00, ...,
        -4.3515575e-01, -8.4614587e-01, -3.9710126e+00],
       ...,
       [ 8.2747601e-02, -6.1710203e-01,  2.3574358e-01, ...,
        -5.8252639e-01,  6.8274313e-01, -1.4978484e+00],
       [-1.2147907e-01,  1.6765369e-01, -9.4606072e-01, ...,
         1.9079995e+00,  4.5488805e-01,  1.4441520e+00],
       [-1.0354982e-01, -1.1097395e-01, -2.1239905e-01, ...,
         6.1147124e-01,  9.5499313e-01, -1.2304487e+00]], dtype=float32)

In [76]:
attri_data

,e,a,v,name_e,name_a,ent_type,minmax,e_idx,a_idx
0,Q593336,P625_Latitude,4.315194e+01,Poumarous,coordinate location(latitude),Q484170,7.397330e-01,38633,0
1,Q511127,P625_Longtiude,3.301111e+00,Nant,coordinate location(logtitude),Q484170,5.068894e-01,34409,1
2,Q59293,P1332_Latitude,4.329426e+01,Porto Sant'Elpidio,coordinates of northernmost point,Q747074,6.482318e-01,22002,2
3,Q100092,P1082,6.460000e+03,San Cesario sul Panaro,population,Q747074,1.551299e-06,81,3
4,Q111736,P2046,1.768000e+07,Sarezzo,area,Q747074,2.903120e-12,4083,4
...,...,...,...,...,...,...,...,...,...
237041,Q128925,P2067,7.500000e+01,Nicolai Boilesen,mass,Q5,3.770587e-29,7688,5
237042,Q6081,P625_Latitude,5.183333e+01,Salzlandkreis,coordinate location(latitude),Q17302772,7.879630e-01,13348,0
237043,Q7612645,P2048,1.880000e+00,Steve Giatzoglou,height,Q5,1.848563e-04,43339,27
237044,Q7234759,P569,1.948000e+03,Ken Murray,date of birth,Q5,9.741294e-01,42386,9


In [79]:
y= attri_data.loc[:,'v'].values

In [77]:
x[:,1]

array([-0.03002811, -0.1611208 ,  0.02539371, ..., -0.617102  ,
        0.1676537 , -0.11097395], dtype=float32)

In [80]:
X_train, y_train = x, y 

In [17]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=802)

In [84]:
nn=MLPRegressor(hidden_layer_sizes=(256),  #預設值1層,數目100
         activation='relu',  #{'identity'(回歸)無，'logistic'為sigmoid，'tanh'，'relu'}，預設='relu'
         solver='adam',   #{'lbfgs'牛頓，'sgd'隨機梯度，'adam'隨機梯度優化}，預設='adam'
         batch_size=32,  #'auto'為min(200～樣本數)
         learning_rate='constant',  #constant固定'，'invscaling隨著時間遞減'，'adaptive誤差減少時不改變'}，預設 ='constant' 
         learning_rate_init=0.001, #優化器為sgd、adam使用
         power_t=0.5,     # 優化器為sgd，則可以設定強化學習率   
         max_iter=50,    #訓練次數
         shuffle=True,    
         random_state=1,   #隨機的基礎值
         momentum=0.9,
        alpha=0.01, #正則
        verbose=True)

In [85]:
nn.fit(X_train, y_train)
y_predtrain=nn.predict(X_train)

Iteration 1, loss = 8340855974373046987853782002678472118332082175499829248.00000000
Iteration 2, loss = 8340855974373046987853782002678472118332082175499829248.00000000
Iteration 3, loss = 8340855974373046987853782002678472118332082175499829248.00000000
Iteration 4, loss = 8340855974373046987853782002678472118332082175499829248.00000000
Iteration 5, loss = 8340855974373046987853782002678472118332082175499829248.00000000
Iteration 6, loss = 8340855974373046987853782002678472118332082175499829248.00000000
Iteration 7, loss = 8340855974373046987853782002678472118332082175499829248.00000000
Iteration 8, loss = 8340855974373045626724314318924618264833652448426983424.00000000
Iteration 9, loss = 8340855974373046987853782002678472118332082175499829248.00000000
Iteration 10, loss = 8340855974373046987853782002678472118332082175499829248.00000000
Iteration 11, loss = 8340855974373045626724314318924618264833652448426983424.00000000
Iteration 12, loss = 834085597437304562672431431892461826483365

/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [83]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import numpy as np

MSE = mean_squared_error(y_train, y_predtrain)
print('MSE=',MSE)
RMSE =np.sqrt(MSE)
print('RMSE=',RMSE)
MAE= mean_absolute_error(y_train, y_predtrain)
print('MAE=',MAE)

R2=1-MSE/np.var(y_predtrain)
print("R2:", R2)

MSE= 1.668171194874609e+55
RMSE= 4.084325152181948e+27
MAE= 8.394835376503991e+24
R2: -4.080808307344429e+43


In [18]:
ytrain = []
for value in y_train:
    ytrain.append(value)
count=0
errvalue=abs((ytrain-y_predtrain)/ytrain)
errvalue

/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """


array([1.94538076e-02, 7.18272868e-03, 9.46137713e-03, ...,
       8.62863626e+00, 7.93208177e-03, 4.10372323e+07])

In [19]:
threshold=0.02  #門檻值
for i in range(len(errvalue)):
  if errvalue[i]<threshold:           
    count=count+1
print("通過門檻筆數",count)
print("通過門檻百分比",count/len(errvalue))

通過門檻筆數 60959
通過門檻百分比 0.25716105734751904


In [61]:
y_pred=nn.predict(X_test)
y_pred

array([ 0.54091746, -0.0991256 ,  0.5813133 , ...,  0.08488109,
       -0.10426868,  0.09598347], dtype=float32)

In [62]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import numpy as np

MSE = mean_squared_error(y_test, y_pred)
print('MSE=',MSE)
RMSE =np.sqrt(MSE)
print('RMSE=',RMSE)
MAE= mean_absolute_error(y_test, y_pred)
print('MAE=',MAE)

R2=1-MSE/np.var(y_test)
print("R2:", R2)

# ytest = []
# for value in y_test:
#     ytest.append(value)
# count=0
# errvalue=abs((ytest-y_pred)/ytest)
# ytest

MSE= 0.00771702871132517
RMSE= 0.08784662037508996
MAE= 0.059130715044269345
R2: 0.946369884237033


In [32]:
threshold=0.01  #門檻值
for i in range(len(errvalue)):
  if errvalue[i]<0.01:           
    count=count+1
print("通過門檻筆數",count)
print("通過門檻百分比",count/len(errvalue))

通過門檻筆數 583
通過門檻百分比 0.019675340015524283


In [47]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.52)#調整alpha值決定正則化的強度
reg.fit(X_train, y_train)
print(reg.coef_) # 印出訓練後的模型參數，數值都明顯小於單純的線性回歸
y_predtrain = reg.predict(X_train)

[-5.16471977e-04  2.01634387e-03  3.67743225e-04 -1.22812157e-03
 -4.09318105e-04 -1.49477576e-03  5.30582329e-04 -5.84212539e-04
 -1.52182067e-04  1.29190285e-03 -8.43106885e-04  1.69864169e-03
  6.54089497e-04  6.58057324e-05 -1.67014761e-04  2.49739358e-04
 -3.46214714e-04 -3.59195331e-03 -3.33673379e-04  3.50631686e-04
 -1.96854788e-04 -1.66668207e-04 -3.85819422e-03  8.19805311e-04
 -5.69804863e-04 -1.13583717e-03 -1.59220910e-03 -3.77230375e-04
 -1.40718024e-04 -1.39299489e-04 -1.74432265e-04  7.74300715e-05
 -3.59961326e-04  9.84100261e-05 -4.76161391e-03  3.17578961e-05
  1.56872778e-03 -2.53054663e-04  2.61779787e-04  7.78805581e-04
 -2.62351800e-03  4.70866187e-04  1.11015863e-03 -8.96150479e-04
  1.49972679e-03 -1.06779533e-02  2.15474446e-03  2.67991738e-04
 -4.20034630e-04 -9.55233932e-04  2.87602801e-04  3.04002780e-04
 -1.76452158e-03  2.52298138e-04  1.96269859e-04 -2.15611211e-03
  5.32028731e-04  3.21957679e-03  1.66951591e-04 -2.14185580e-04
 -5.62372501e-04 -1.13395

/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.47092e-08): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [42]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import numpy as np

MSE = mean_squared_error(y_train, y_predtrain)
print('MSE=',MSE)
RMSE =np.sqrt(MSE)
print('RMSE=',RMSE)
MAE= mean_absolute_error(y_train, y_predtrain)
print('MAE=',MAE)

R2=1-MSE/np.var(y_predtrain)
print("R2:", R2)

MSE= 3.220578710718283e+40
RMSE= 1.7945970886854472e+20
MAE= 6.87268669111927e+18
R2: 1.0


/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/numpy/core/_methods.py:233: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/numpy/core/_methods.py:244: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [49]:
from sklearn.linear_model import RidgeCV
rcv = RidgeCV(alphas=np.array([.6, .7, .8, .9]))
rcv.fit(X_train, y_train)
#RidgeCV(alphas=array([0.1, 0.2, 0.3, 0.4]), cv=None, fit_intercept=True,
       # gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)
print(rcv.alpha_)

0.6


In [50]:
rcv2 = RidgeCV(alphas=np.array([.63, .64, .65, .66, .67]))
rcv2.fit(X_train, y_train)
print(rcv2.alpha_)

0.63


In [51]:
alphas_to_test = np.linspace(0.01, 1)
rcv3 = RidgeCV(alphas=alphas_to_test, store_cv_values=True)
rcv3.fit(X_train, y_train)

RidgeCV(alphas=array([0.01      , 0.03020408, 0.05040816, 0.07061224, 0.09081633,
       0.11102041, 0.13122449, 0.15142857, 0.17163265, 0.19183673,
       0.21204082, 0.2322449 , 0.25244898, 0.27265306, 0.29285714,
       0.31306122, 0.33326531, 0.35346939, 0.37367347, 0.39387755,
       0.41408163, 0.43428571, 0.4544898 , 0.47469388, 0.49489796,
       0.51510204, 0.53530612, 0.5555102 , 0.57571429, 0.59591837,
       0.61612245, 0.63632653, 0.65653061, 0.67673469, 0.69693878,
       0.71714286, 0.73734694, 0.75755102, 0.7777551 , 0.79795918,
       0.81816327, 0.83836735, 0.85857143, 0.87877551, 0.89897959,
       0.91918367, 0.93938776, 0.95959184, 0.97979592, 1.        ]),
        store_cv_values=True)

In [47]:
rcv3.cv_values_.shape

(122474, 50)

In [52]:
def MAD(target, predictions):
    absolute_deviation = np.abs(target - predictions)
    return absolute_deviation.mean()

In [53]:
import sklearn
MAD = sklearn.metrics.make_scorer(MAD, greater_is_better=False)
rcv4 = RidgeCV(alphas=alphas_to_test, store_cv_values=True,scoring=MAD)
rcv4.fit(X_train, y_train)
smallest_idx = rcv4.cv_values_.mean(axis=0).argmin()
alphas_to_test[smallest_idx]

0.01